The following additional libraries are needed to run this
notebook. Note that running on Colab is experimental, please report a Github
issue if you have any problem.

In [1]:
!pip install d2l==1.0.3


# Multiple Input and Multiple Output Channels
:label:`sec_channels`

While we described the multiple channels
that comprise each image (e.g., color images have the standard RGB channels
to indicate the amount of red, green and blue) and convolutional layers for multiple channels in :numref:`subsec_why-conv-channels`,
until now, we simplified all of our numerical examples
by working with just a single input and a single output channel.
This allowed us to think of our inputs, convolution kernels,
and outputs each as two-dimensional tensors.

When we add channels into the mix,
our inputs and hidden representations
both become three-dimensional tensors.
For example, each RGB input image has shape $3\times h\times w$.
We refer to this axis, with a size of 3, as the *channel* dimension. The notion of
channels is as old as CNNs themselves: for instance LeNet-5 :cite:`LeCun.Jackel.Bottou.ea.1995` uses them.
In this section, we will take a deeper look
at convolution kernels with multiple input and multiple output channels.


In [2]:
import torch
from d2l import torch as d2l

## Multiple Input Channels

When the input data contains multiple channels,
we need to construct a convolution kernel
with the same number of input channels as the input data,
so that it can perform cross-correlation with the input data.
Assuming that the number of channels for the input data is $c_\textrm{i}$,
the number of input channels of the convolution kernel also needs to be $c_\textrm{i}$. If our convolution kernel's window shape is $k_\textrm{h}\times k_\textrm{w}$,
then, when $c_\textrm{i}=1$, we can think of our convolution kernel
as just a two-dimensional tensor of shape $k_\textrm{h}\times k_\textrm{w}$.

However, when $c_\textrm{i}>1$, we need a kernel
that contains a tensor of shape $k_\textrm{h}\times k_\textrm{w}$ for *every* input channel. Concatenating these $c_\textrm{i}$ tensors together
yields a convolution kernel of shape $c_\textrm{i}\times k_\textrm{h}\times k_\textrm{w}$.
Since the input and convolution kernel each have $c_\textrm{i}$ channels,
we can perform a cross-correlation operation
on the two-dimensional tensor of the input
and the two-dimensional tensor of the convolution kernel
for each channel, adding the $c_\textrm{i}$ results together
(summing over the channels)
to yield a two-dimensional tensor.
This is the result of a two-dimensional cross-correlation
between a multi-channel input and
a multi-input-channel convolution kernel.

:numref:`fig_conv_multi_in` provides an example
of a two-dimensional cross-correlation with two input channels.
The shaded portions are the first output element
as well as the input and kernel tensor elements used for the output computation:
$(1\times1+2\times2+4\times3+5\times4)+(0\times0+1\times1+3\times2+4\times3)=56$.

![Cross-correlation computation with two input channels.](https://github.com/d2l-ai/d2l-pytorch-colab/blob/master/img/conv-multi-in.svg?raw=1)
:label:`fig_conv_multi_in`


To make sure we really understand what is going on here,
we can (**implement cross-correlation operations with multiple input channels**) ourselves.
Notice that all we are doing is performing a cross-correlation operation
per channel and then adding up the results.


In [3]:
def corr2d_multi_in(X, K):
    # Iterate through the 0th dimension (channel) of K first, then add them up
    return sum(d2l.corr2d(x, k) for x, k in zip(X, K))

We can construct the input tensor `X` and the kernel tensor `K`
corresponding to the values in :numref:`fig_conv_multi_in`
to (**validate the output**) of the cross-correlation operation.


In [4]:
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
               [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

## Multiple Output Channels
:label:`subsec_multi-output-channels`

Regardless of the number of input channels,
so far we always ended up with one output channel.
However, as we discussed in :numref:`subsec_why-conv-channels`,
it turns out to be essential to have multiple channels at each layer.
In the most popular neural network architectures,
we actually increase the channel dimension
as we go deeper in the neural network,
typically downsampling to trade off spatial resolution
for greater *channel depth*.
Intuitively, you could think of each channel
as responding to a different set of features.
The reality is a bit more complicated than this. A naive interpretation would suggest
that representations are learned independently per pixel or per channel.
Instead, channels are optimized to be jointly useful.
This means that rather than mapping a single channel to an edge detector, it may simply mean
that some direction in channel space corresponds to detecting edges.

Denote by $c_\textrm{i}$ and $c_\textrm{o}$ the number
of input and output channels, respectively,
and by $k_\textrm{h}$ and $k_\textrm{w}$ the height and width of the kernel.
To get an output with multiple channels,
we can create a kernel tensor
of shape $c_\textrm{i}\times k_\textrm{h}\times k_\textrm{w}$
for *every* output channel.
We concatenate them on the output channel dimension,
so that the shape of the convolution kernel
is $c_\textrm{o}\times c_\textrm{i}\times k_\textrm{h}\times k_\textrm{w}$.
In cross-correlation operations,
the result on each output channel is calculated
from the convolution kernel corresponding to that output channel
and takes input from all channels in the input tensor.

We implement a cross-correlation function
to [**calculate the output of multiple channels**] as shown below.


In [5]:
def corr2d_multi_in_out(X, K):
    # Iterate through the 0th dimension of K, and each time, perform
    # cross-correlation operations with input X. All of the results are
    # stacked together
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)

We construct a trivial convolution kernel with three output channels
by concatenating the kernel tensor for `K` with `K+1` and `K+2`.


In [6]:
K = torch.stack((K, K + 1, K + 2), 0)
K.shape

torch.Size([3, 2, 2, 2])

Below, we perform cross-correlation operations
on the input tensor `X` with the kernel tensor `K`.
Now the output contains three channels.
The result of the first channel is consistent
with the result of the previous input tensor `X`
and the multi-input channel,
single-output channel kernel.


In [7]:
corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

## $1\times 1$ Convolutional Layer
:label:`subsec_1x1`

At first, a [**$1 \times 1$ convolution**], i.e., $k_\textrm{h} = k_\textrm{w} = 1$,
does not seem to make much sense.
After all, a convolution correlates adjacent pixels.
A $1 \times 1$ convolution obviously does not.
Nonetheless, they are popular operations that are sometimes included
in the designs of complex deep networks :cite:`Lin.Chen.Yan.2013,Szegedy.Ioffe.Vanhoucke.ea.2017`.
Let's see in some detail what it actually does.

Because the minimum window is used,
the $1\times 1$ convolution loses the ability
of larger convolutional layers
to recognize patterns consisting of interactions
among adjacent elements in the height and width dimensions.
The only computation of the $1\times 1$ convolution occurs
on the channel dimension.

:numref:`fig_conv_1x1` shows the cross-correlation computation
using the $1\times 1$ convolution kernel
with 3 input channels and 2 output channels.
Note that the inputs and outputs have the same height and width.
Each element in the output is derived
from a linear combination of elements *at the same position*
in the input image.
You could think of the $1\times 1$ convolutional layer
as constituting a fully connected layer applied at every single pixel location
to transform the $c_\textrm{i}$ corresponding input values into $c_\textrm{o}$ output values.
Because this is still a convolutional layer,
the weights are tied across pixel location.
Thus the $1\times 1$ convolutional layer requires $c_\textrm{o}\times c_\textrm{i}$ weights
(plus the bias). Also note that convolutional layers are typically followed
by nonlinearities. This ensures that $1 \times 1$ convolutions cannot simply be
folded into other convolutions.

![The cross-correlation computation uses the $1\times 1$ convolution kernel with three input channels and two output channels. The input and output have the same height and width.](https://github.com/d2l-ai/d2l-pytorch-colab/blob/master/img/conv-1x1.svg?raw=1)
:label:`fig_conv_1x1`

Let's check whether this works in practice:
we implement a $1 \times 1$ convolution
using a fully connected layer.
The only thing is that we need to make some adjustments
to the data shape before and after the matrix multiplication.


In [8]:
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h * w))
    K = K.reshape((c_o, c_i))
    # Matrix multiplication in the fully connected layer
    Y = torch.matmul(K, X)
    return Y.reshape((c_o, h, w))

When performing $1\times 1$ convolutions,
the above function is equivalent to the previously implemented cross-correlation function `corr2d_multi_in_out`.
Let's check this with some sample data.


In [9]:
X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(0, 1, (2, 3, 1, 1))
Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
assert float(torch.abs(Y1 - Y2).sum()) < 1e-6

## Discussion

Channels allow us to combine the best of both worlds: MLPs that allow for significant nonlinearities and convolutions that allow for *localized* analysis of features. In particular, channels allow the CNN to reason with multiple features, such as edge and shape detectors at the same time. They also offer a practical trade-off between the drastic parameter reduction arising from translation invariance and locality, and the need for expressive and diverse models in computer vision.

Note, though, that this flexibility comes at a price. Given an image of size $(h \times w)$, the cost for computing a $k \times k$ convolution is $\mathcal{O}(h \cdot w \cdot k^2)$. For $c_\textrm{i}$ and $c_\textrm{o}$ input and output channels respectively this increases to $\mathcal{O}(h \cdot w \cdot k^2 \cdot c_\textrm{i} \cdot c_\textrm{o})$. For a $256 \times 256$ pixel image with a $5 \times 5$ kernel and $128$ input and output channels respectively this amounts to over 53 billion operations (we count multiplications and additions separately). Later on we will encounter effective strategies to cut down on the cost, e.g., by requiring the channel-wise operations to be block-diagonal, leading to architectures such as ResNeXt :cite:`Xie.Girshick.Dollar.ea.2017`.

## Exercises

1. Assume that we have two convolution kernels of size $k_1$ and $k_2$, respectively
   (with no nonlinearity in between).
    1. Prove that the result of the operation can be expressed by a single convolution.
    1. What is the dimensionality of the equivalent single convolution?
    1. Is the converse true, i.e., can you always decompose a convolution into two smaller ones?
1. Assume an input of shape $c_\textrm{i}\times h\times w$ and a convolution kernel of shape
   $c_\textrm{o}\times c_\textrm{i}\times k_\textrm{h}\times k_\textrm{w}$, padding of $(p_\textrm{h}, p_\textrm{w})$, and stride of $(s_\textrm{h}, s_\textrm{w})$.
    1. What is the computational cost (multiplications and additions) for the forward propagation?
    1. What is the memory footprint?
    1. What is the memory footprint for the backward computation?
    1. What is the computational cost for the backpropagation?
1. By what factor does the number of calculations increase if we double both the number of input channels
   $c_\textrm{i}$ and the number of output channels $c_\textrm{o}$? What happens if we double the padding?
1. Are the variables `Y1` and `Y2` in the final example of this section exactly the same? Why?
1. Express convolutions as a matrix multiplication, even when the convolution window is not $1 \times 1$.
1. Your task is to implement fast convolutions with a $k \times k$ kernel. One of the algorithm candidates
   is to scan horizontally across the source, reading a $k$-wide strip and computing the $1$-wide output strip
   one value at a time. The alternative is to read a $k + \Delta$ wide strip and compute a $\Delta$-wide
   output strip. Why is the latter preferable? Is there a limit to how large you should choose $\Delta$?
1. Assume that we have a $c \times c$ matrix.
    1. How much faster is it to multiply with a block-diagonal matrix if the matrix is broken up into $b$ blocks?
    1. What is the downside of having $b$ blocks? How could you fix it, at least partly?


[Discussions](https://discuss.d2l.ai/t/70)


### Exercise 1:

#### 1.1.
Given an input tensor $ x $ and a kernel $ k_1 $, the **cross-correlation** is defined as:

$$
(y_1)_t = (x \star k_1)_t = \sum_{s \in S} x_{t+s} k_{1s}
$$

where:
- $ x_t $ is the input signal (or tensor).
- $ k_{1s} $ is the first kernel.
- The sum runs over all valid indices $ s $ where the kernel is defined.

Now, applying a second kernel $ k_2 $ to $ y_1 $:

$$
(y_2)_u = (y_1 \star k_2)_u = \sum_{s \in S} y_{1,u+s} k_{2s}
$$

Substituting $ y_{1,u+s} $ from the first step:

$$
(y_2)_u = \sum_{s \in S} \left( \sum_{t \in S} x_{u+s+t} k_{1t} \right) k_{2s}
$$

Rearranging the sums:

$$
(y_2)_u = \sum_{t \in S} \sum_{s \in S} x_{u+s+t} k_{1t} k_{2s}
$$

Recognizing the form of cross-correlation, we define an **equivalent kernel**:

$4
w_{\text{eq}, v} = \sum_{s \in S} k_{1,v-s} k_{2s}
$$

so that:

$$
(y_2)_u = \sum_{v \in S} x_{u+v} w_{\text{eq}, v}
$$

which is precisely a **single cross-correlation** operation:

$$
y_2 = x \star w_{\text{eq}}
$$

The equivalent kernel $ w_{\text{eq}} $ is the **cross-correlation** of $ k_1 $ and $ k_2 $:

$$
w_{\text{eq}} = k_1 \star k_2
$$



#### 1.2.
The dimensionality of the resulting single convolution is $k_1 + k_2 - 1$

#### 1.4.

The converse would state that any convolution with kernel size $k_{eq}$ can always be decomposed into two smaller convolutions of sizes $k_1$ and $k_2$.

In general, this is not always possible. The decomposition depends on the structure of the kernel:

- If the kernel is separable, meaning it can be factored into two smaller kernels, then decomposition is possible.
- However, many kernels are non-separable and do not allow exact decomposition into smaller convolutions.

### Exercise 2:

#### 2.1.

$$c_o \times c_i \times k_h \times k_w \times [(h+p_h-k_h+s_h)//s_h] \times [(w+p_w-k_w+s_w)//s_w]$$

### Exercise 2:

We assume an input of shape $ c_i \times h \times w $ and a convolution kernel of shape $ c_o \times c_i \times k_h \times k_w $, with padding $ (p_h, p_w) $ and stride $ (s_h, s_w) $.

#### **1.1 Forward Propagation Computational Cost**
The **output size** is determined by:

$$
h' = \frac{h + 2p_h - k_h}{s_h} + 1, \quad w' = \frac{w + 2p_w - k_w}{s_w} + 1
$$

Each output value requires a **dot product** between the kernel and the corresponding input patch. Each dot product involves:

$$
c_i \times k_h \times k_w
$$

multiplications and an equal number of additions.

Since we compute $ c_o $ output channels, and there are $ h' \times w' $ spatial positions, the total number of multiplications and additions is:

$$
\text{Multiplications} = h' w' c_o \cdot (c_i k_h k_w)
$$

$$
\text{Additions} = h' w' c_o \cdot (c_i k_h k_w - 1)
$$

#### **1.2 Forward Memory Footprint**
Memory is needed for:
- Input: $ c_i \times h \times w $
- Kernel: $ c_o \times c_i \times k_h \times k_w $
- Output: $ c_o \times h' \times w' $

Thus, the total memory footprint is:

$$
c_i h w + c_o c_i k_h k_w + c_o h' w'
$$

#### **1.3 Backward Memory Footprint**
For backpropagation, we need to store:
- Input $ x $
- Gradients $ \frac{\partial L}{\partial x} $ (same size as input)
- Gradients $ \frac{\partial L}{\partial w} $ (same size as kernel)
- Gradients $ \frac{\partial L}{\partial y} $ (same size as output)

Thus, the **backward memory footprint** is:

$$
2(c_i h w) + (c_o h' w') + (c_o c_i k_h k_w)
$$

#### **1.4 Computational Cost for Backpropagation**
Backpropagation consists of:
1. **Gradient w.r.t. input**: This requires convolving $ \frac{\partial L}{\partial y} $ with a **flipped kernel**, leading to:

   $$
   h w c_i \cdot (c_o k_h k_w)
   $$

   multiplications.

2. **Gradient w.r.t. kernel**: This involves **convolving** $ x $ with $ \frac{\partial L}{\partial y} $, leading to:

   $$
   c_o c_i k_h k_w \cdot (h' w')
   $$

   multiplications.

Thus, the total cost is:

$$
\text{Multiplications} = h w c_i (c_o k_h k_w) + c_o c_i k_h k_w (h' w')
$$

---

### **3. Effect of Doubling $ c_i $, $ c_o $, and Padding**
#### **Doubling $ c_i $ and $ c_o $**
If both $ c_i $ and $ c_o $ are doubled:
- Each kernel grows by $ 2 \times 2 = 4 $.
- The output still requires processing $ h' \times w' $ patches.

The computational cost increases by a factor of **4**:

$$
\text{New Cost} = 4 \times \text{Original Cost}
$$

#### **Doubling Padding**
Padding affects output size:

$$
h' = \frac{h + 4p_h - k_h}{s_h} + 1, \quad w' = \frac{w + 4p_w - k_w}{s_w} + 1
$$

If padding increases but kernel and stride remain the same, the **output size grows**, increasing computations.



### **4. Are $ Y_1 $ and $ Y_2 $ the Same?**
Not exactly the same, because the theoretical equivalence holds when the operations are performed with ideal precision and mathematical properties. In practice, especially with finite precision computations, slight differences may arise due to numerical limitations and implementation details.



In [11]:
Y1, Y2

(tensor([[[ 1.1545e+00,  2.7878e+00,  1.2037e+00],
          [-1.0400e+00,  2.0920e+00,  6.2781e-01],
          [-6.1205e+00,  4.2483e-03, -4.1907e+00]],
 
         [[ 2.4859e+00,  4.0840e-01, -2.6984e+00],
          [ 1.7975e+00, -3.2594e+00,  5.3007e-01],
          [ 1.8862e+00,  5.1405e-02,  3.4953e+00]]]),
 tensor([[[ 1.1545e+00,  2.7878e+00,  1.2037e+00],
          [-1.0400e+00,  2.0920e+00,  6.2781e-01],
          [-6.1205e+00,  4.2483e-03, -4.1907e+00]],
 
         [[ 2.4859e+00,  4.0840e-01, -2.6984e+00],
          [ 1.7975e+00, -3.2594e+00,  5.3007e-01],
          [ 1.8862e+00,  5.1405e-02,  3.4953e+00]]]))

---

### **5. Expressing Convolution as Matrix Multiplication**
A convolution can be rewritten as a matrix multiplication using the **im2col** trick:
1. Unroll input patches into a matrix $ X' $ of shape $ (h' w', c_i k_h k_w) $.
2. Reshape kernels into a matrix $ W' $ of shape $ (c_o, c_i k_h k_w) $.
3. Perform matrix multiplication:

   $$
   Y' = W' X'
   $$

where $ Y' $ is reshaped back into $ (c_o, h', w') $.

---

### **6. Fast Convolutions with a $ k \times k $ Kernel**
Instead of scanning a **1-wide strip**, we can process a **$ k + \Delta $-wide strip**:

- **Advantage**: This reduces redundant memory access and increases cache efficiency.
- **Limit**: Large $ \Delta $ may increase memory overhead and require additional buffering.

A good choice of $ \Delta $ is system-dependent, often optimizing for **cache utilization**.

---

### **7. Block-Diagonal Matrices for Fast Multiplication**
#### **6.1 Speedup with $ b $ Blocks**
If a matrix is split into $ b $ **block-diagonal** parts, the speedup is:

$$
\frac{c^3}{b (c/b)^3} = b^2
$$

Multiplication is reduced from $ O(c^3) $ to $ O(b (c/b)^3) = O(c^3/b^2) $.

#### **6.2 Downside of Many Blocks**
- Increasing $ b $ reduces computational cost **but** increases **communication overhead** between blocks.
- Solution: Use **overlapping blocks** or **hierarchical block matrices**.